In [1]:
import analysis
import pickle
import pandas as pd
import numpy as np

In [2]:
df_All = analysis.getAllExperiments()

coriRange = list( range( 85, 235 ) ) + list( range( 330, 450 ) )
df_Cori = df_All.loc[ df_All[ 'Experiment' ].isin( coriRange ) ]

attawayRange = list( range( 235, 330 ) ) + list( range( 450, 550 ) )
df_Attaway = df_All.loc[ df_All[ 'Experiment' ].isin( attawayRange ) ]

In [4]:
ppn = 32

for workload in ['sleep', 'fwq', 'dgemm', 'spmv', 'hpcg', 'lammps']:
    df_Cori_256_NoStencil = df_Cori[(df_Cori['workload'] == workload) & (df_Cori['processors'] == 256) & (df_Cori['stencil_size'] == 0) & (df_Cori['rabbit_workload'] == 0)]
    
    if workload == 'lammps':
        df_Cori_2048_NoStencil = df_Cori[(df_Cori['workload'] == workload) & (df_Cori['processors'] == 1728) & (df_Cori['stencil_size'] == 0) & (df_Cori['rabbit_workload'] == 0)]
    else:
        df_Cori_2048_NoStencil = df_Cori[(df_Cori['workload'] == workload) & (df_Cori['processors'] == 2048) & (df_Cori['stencil_size'] == 0) & (df_Cori['rabbit_workload'] == 0)]
    
    Cori_256 = analysis.getData(df_Cori_256_NoStencil.iloc[2])
    Cori_256 = Cori_256[['workload', 'node', 'rank', 'iteration', 'iterations', 'workload_usec']]
    Cori_256_Iteration0 = Cori_256[(Cori_256['iteration'] == 0)]
    
    print(Cori_256.head())
    
    nodes_256 =  Cori_256['node'].unique()
    node_counter = np.zeros((len(nodes_256), 1))
    
    iterations = Cori_256['iterations'].iloc[0]
    data_256 = np.zeros((int(256 / ppn), ppn, iterations))
    
    for item in range(0, len(Cori_256_Iteration0)):
        cur = Cori_256_Iteration0.iloc[item]
        
        for node in nodes_256:
            if node == cur['node']:
                i = int(np.where(nodes_256 == node)[0][0])
                j = int(node_counter[i])
        node_counter[i] = node_counter[i] + 1
        
        curData = Cori_256[Cori_256['rank'] == cur['rank']]
        
        for k in range(0, iterations):
            temp = curData.iloc[k]
            data_256[i, j, k] = temp['workload_usec']
    
    fname = 'Trilce/Cori_' + workload + '_256_2.npy'
    np.save(fname, data_256)
    
    Cori_2048 = analysis.getData(df_Cori_2048_NoStencil.iloc[2])
    Cori_2048 = Cori_2048[['workload', 'node', 'rank', 'iteration', 'iterations', 'workload_usec']]
    Cori_2048_Iteration0 = Cori_2048[(Cori_2048['iteration'] == 0)]
    
    print(Cori_2048.head())
    
    nodes_2048 =  Cori_2048['node'].unique()
    node_counter = np.zeros((len(nodes_2048), 1))
    
    iterations = Cori_2048['iterations'].iloc[0]
    data_2048 = np.zeros((int(2048 / ppn), ppn, iterations))
    
    for item in range(0, len(Cori_2048_Iteration0)):
        cur = Cori_2048_Iteration0.iloc[item]
        
        for node in nodes_2048:
            if node == cur['node']:
                i = int(np.where(nodes_2048 == node)[0][0])
                j = int(node_counter[i])
        node_counter[i] = node_counter[i] + 1
        
        curData = Cori_2048[Cori_2048['rank'] == cur['rank']]
        
        for k in range(0, iterations):
            data_2048[i, j, k] = curData['workload_usec'].iloc[k]
           
    fname = 'Trilce/Cori_' + workload + '_2048_2.npy'
    np.save(fname, data_2048)

  workload      node  rank  iteration  iterations  workload_usec
0    sleep  nid00865     0          0         500      93685.304
1    sleep  nid00865     0          1         500      92806.052
2    sleep  nid00865     0          2         500     100891.835
3    sleep  nid00865     0          3         500     103619.767
4    sleep  nid00865     0          4         500      95238.270
  workload      node  rank  iteration  iterations  workload_usec
0    sleep  nid00790     0          0         500      84811.597
1    sleep  nid00790     0          1         500      99383.354
2    sleep  nid00790     0          2         500     115466.795
3    sleep  nid00790     0          3         500      95301.253
4    sleep  nid00790     0          4         500     104439.699
  workload      node  rank  iteration  iterations  workload_usec
0      fwq  nid01882     0          0         500      95399.275
1      fwq  nid01882     0          1         500     100805.005
2      fwq  nid01882     